In [ ]:
"""
CNN for CIFAR-10 – fixed & robust
TensorFlow/Keras 2.17+ • Python 3.10+
"""
import os, sys, time
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks
import matplotlib.pyplot as plt

# ------------------------------------
# 1. Load & preprocess CIFAR-10
# ------------------------------------
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

# Convert to float32 *before* normalising
x_train = x_train.astype("float32") / 255.0
x_test  = x_test.astype("float32")  / 255.0

# Flatten labels to shape (n,) and cast to int32
y_train = y_train.flatten().astype("int32")
y_test  = y_test.flatten().astype("int32")

# ------------------------------------
# 2. Build the CNN
# ------------------------------------
model = models.Sequential([
    layers.Conv2D(32, 3, activation="relu", padding="same",
                  input_shape=(32, 32, 3)),
    layers.MaxPooling2D(),

    layers.Conv2D(64, 3, activation="relu", padding="same"),
    layers.MaxPooling2D(),

    layers.Conv2D(128, 3, activation="relu", padding="same"),
    layers.MaxPooling2D(),

    layers.Flatten(),
    layers.Dense(128, activation="relu"),
    layers.Dropout(0.3),
    layers.Dense(10, activation="softmax")
])

model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()

# ------------------------------------
# 3. Train (with early stopping)
# ------------------------------------
early_stop = callbacks.EarlyStopping(
    monitor="val_accuracy",
    patience=3,
    restore_best_weights=True
)

history = model.fit(
    x_train, y_train,
    epochs=30,
    batch_size=64,
    validation_split=0.2,
    callbacks=[early_stop],
    verbose=2
)

# ------------------------------------
# 4. Evaluate
# ------------------------------------
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)
print(f"\n🧪  Test accuracy: {test_acc:.4f}")

# ------------------------------------
# 5. Plot training history (skip when running headless)
# ------------------------------------
if sys.stdin.isatty():  # crude check: only plot in interactive sessions
    plt.figure(figsize=(8,5))
    plt.plot(history.history["accuracy"], label="Train")
    plt.plot(history.history["val_accuracy"], label="Val")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.title("Training vs. Validation Accuracy")
    plt.legend()
    plt.tight_layout()
    plt.show()

# ------------------------------------
# 6. Save the model
# ------------------------------------
model.save("cnn_cifar10_model.h5")
print("✅  Model saved as cnn_cifar10_model.h5")

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 32, 32, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 16, 16, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 8, 8, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 4, 4, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │       262,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 356,810 (1.36 MB)

 Trainable params: 356,810 (1.36 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/30
625/625 - 119s - 190ms/step - accuracy: 0.4069 - loss: 1.6199 - val_accuracy: 0.5481 - val_loss: 1.2672
Epoch 2/30
625/625 - 139s - 222ms/step - accuracy: 0.5702 - loss: 1.2130 - val_accuracy: 0.6320 - val_loss: 1.0464
Epoch 3/30
625/625 - 113s - 180ms/step - accuracy: 0.6358 - loss: 1.0268 - val_accuracy: 0.6790 - val_loss: 0.9163
Epoch 4/30
625/625 - 117s - 188ms/step - accuracy: 0.6827 - loss: 0.9134 - val_accuracy: 0.7056 - val_loss: 0.8434
Epoch 5/30
625/625 - 140s - 223ms/step - accuracy: 0.7127 - loss: 0.8213 - val_accuracy: 0.7184 - val_loss: 0.8028
Epoch 6/30
625/625 - 116s - 186ms/step - accuracy: 0.7394 - loss: 0.7465 - val_accuracy: 0.7226 - val_loss: 0.7917
Epoch 7/30
625/625 - 113s - 180ms/step - accuracy: 0.7636 - loss: 0.6724 - val_accuracy: 0.7298 - val_loss: 0.7787
Epoch 8/30
625/625 - 145s - 232ms/step - accuracy: 0.7786 - loss: 0.6244 - val_accuracy: 0.7363 - val_loss: 0.7707
Epoch 9/30
625/625 - 116s - 186ms/step - accuracy: 0.7980 - loss: 0.5733 - val_a